In [1]:
# coding: utf-8
from pyspark.sql import Window, Row
from pyspark.sql.functions import col, count
from pyspark.sql.functions import rank, row_number
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.types import LongType

import redis
import pandas as pd
import app_constants
import spark_setup
import database_setup

# Setups
db = database_setup.DatabaseSetup.db
spark = spark_setup.SparkSetup.spark

In [2]:
# Loading data as spark DF. Column with dots -> Upgrade pyspark > 2.0.0
session_df = spark.read \
                .option("delimiter", ",") \
                .option("inferSchema", "true") \
                .option("header", spark_setup.SparkSetup.data_headers) \
                .csv('data/test.csv')

# Data clean up: REMOVE duplicate records and if required column has NULL value
clean_df = session_df.drop_duplicates() \
                    .dropna(subset=app_constants.Columns.REQUIRED) \
                    .select(app_constants.Columns.REQUIRED)

In [30]:
date_window = Window.partitionBy(clean_df.dateAdded) \
                    .orderBy(clean_df.dateAdded.desc(),
                            clean_df.dateUpdated.desc())

recent_df = clean_df.withColumn('date_timestamp',
                          unix_timestamp(clean_df.dateAdded.cast('date'))) \
                    .withColumn("date_added", unix_timestamp(clean_df.dateAdded)) \
                    .withColumn("date_updated", unix_timestamp(clean_df.dateUpdated)) \
                    .withColumn('row_number', row_number().over(date_window)) \
                    .filter(col('row_number') == app_constants.Count.RECENT_DATA) \
                    .drop('row_number', 'dateAdded', 'dateUpdated') \
                    .withColumnRenamed('date_added', 'dateAdded') \
                    .withColumnRenamed('date_updated', 'dateUpdated')

# recent_dict = recent_df.toPandas().to_dict('records')

# for data in recent_dict:
#     # Create key with recent namespace & epoch date
#     recent_key = app_constants.KeyMeta.RECENT + app_constants.KeyMeta.JOINER + str(data['date_timestamp'])
#     # Assigning entire hash to the key
#     db.hmset(recent_key, data)

In [31]:
# recent_dict[0]['date_timestamp'] # datetime.date(2015, 10, 17), 1445020200
recent_df.show()

+--------------------+------------+------------------+--------------+----------+-----------+
|                  id|       brand|            colors|date_timestamp| dateAdded|dateUpdated|
+--------------------+------------+------------------+--------------+----------+-----------+
|AVpe_3ai1cnluZ0-boTX|Michael Kors|             Black|    1457029800|1457058929| 1457058929|
|AVpe_0ob1cnluZ0-bnWr|  BEAUTIFEET|  Multicolor,MULTI|    1478197800|1478229071| 1478229071|
|AVpe_54gilAPnD_xSr-B| Faded Glory|          TAN,Grey|    1455820200|1455829062| 1485749520|
|AVpe_1O_LJeJML430GGC|        Lugz|             Brown|    1430677800|1430742012| 1488472613|
|AVpe_4ijilAPnD_xSrfB|   Wild Diva|             Brown|    1442773800|1442838209| 1488814919|
|AVpe_19KilAPnD_xSqnY|    Caparros|            Silver|    1478197800|1478227840| 1478227840|
|AVpe_4OLLJeJML430HJo|    SKECHERS|             Black|    1478802600|1478857876| 1478857876|
|AVpe_34HilAPnD_xSrP8|       Bloch|White,Black,Silver|    1483986600|1

In [111]:
# import json

# recent_hash = db.Hash('recent:100')
# recent_key = db.get_key('recent:100')
# recent_hash.update(data)
# eval(db.__getitem__('recent:100'))
# print recent_hash

# data = recent_dict[1]
# data = json.dumps(recent_dict[0])
# db.set('recent:200', data)
# db.get('recent:200')

# db.hmset('recent:300', data)
# db.hgetall('recent:1445020200')

# type(json.dumps([{'x': 2}, {'x': 3}]))
# db.get('recent:1445020200')

{'brand': 'JingXiGuoJi',
 'colors': 'White,Red,Black,Gold,Yellow,Blue,Orange,Purple',
 'dateAdded': '1445022595',
 'dateUpdated': '1468917954',
 'date_timestamp': '1445020200',
 'id': 'AVpe5Zm4ilAPnD_xQVqz'}

In [138]:
count_df = clean_df.withColumn('date_timestamp', \
                        unix_timestamp(clean_df.dateAdded.cast('date'))) \
                  .groupBy('date_timestamp', 'brand') \
                  .agg(count('brand')) \
                  .orderBy('date_timestamp', 'count(brand)', ascending=False)

count_dict = count_df.toPandas() \
                  .groupby('date_timestamp') \
                  .apply(lambda x: dict(zip(x['brand'], x['count(brand)']))) \
                  .to_dict()

for epoch_date, data in count_dict.iteritems():
    # Create key with count namespace & epoch date
    count_key = app_constants.KeyMeta.COUNT + app_constants.KeyMeta.JOINER + str(epoch_date)
    # Assigning entire hash to the redis key
    db.hmset(count_key, data)

In [139]:
count_dict

{1430677800: {u'Lugz': 1},
 1442773800: {u'Wild Diva': 1},
 1455820200: {u'Faded Glory': 1},
 1457029800: {u'Michael Kors': 1},
 1478197800: {u'BEAUTIFEET': 1, u'Caparros': 1},
 1478802600: {u'SKECHERS': 1},
 1483986600: {u'Bloch': 1},
 1485455400: {u'Vince Camuto': 2},
 1486146600: {u'Michael Kors': 1, u'Novica': 3}}

In [59]:
color_window = Window.partitionBy(clean_df.colors) \
                    .orderBy(clean_df.dateAdded.desc(), \
                            clean_df.dateUpdated.desc())

color_df = clean_df.select('*', row_number() \
                                .over(color_window) \
                                .alias('row_number')) \
                    .filter(col('row_number') <= app_constants.Count.COLOR_DATA) \
                    .withColumn('dateAdded', unix_timestamp(clean_df.dateAdded).cast(LongType())) \
                    .withColumn('dateUpdated', unix_timestamp(clean_df.dateUpdated).cast(LongType())) \
                    .drop('row_number')

color_dict = color_df.toPandas() \
                    .groupby(['colors']) \
                    .apply(lambda x: x.to_dict('records'))

# for k, v in color_dict.iteritems():
#     for color in k.split(','):
#         key = (app_constants.KeyMeta.COLOR + app_constants.KeyMeta.JOINER + color).lower()
#         if db.exists(key): db.delete(key)

# for color, data in color_dict.iteritems():
#     split_colors = color.split(',')

#     for split_color in split_colors:
#         color_key = (app_constants.KeyMeta.COLOR + app_constants.KeyMeta.JOINER + split_color).lower()
#         db.rpush(color_key, *data)

In [66]:
# color_df.show()

# key = 'color:100'
# import json
data = color_dict[0][0]
data
# db.delete(key)
# db.exists(key)
# db.rpush(key, *data)
# db.lrange(key, 0, -1)

# for k, v in color_dict.iteritems():
#     for color in k.split(','):
#         key = (app_constants.KeyMeta.COLOR + app_constants.KeyMeta.JOINER + color).lower()
#         if db.exists(key): db.delete(key)

# for color, data in color_dict.iteritems():
#     split_colors = color.split(',')

#     for split_color in split_colors:
#         color_key = (app_constants.KeyMeta.COLOR + app_constants.KeyMeta.JOINER + split_color).lower()
#         db.rpush(color_key, *data)

# print db.lrange('color:silver', 0, -1)

{'brand': u'SKECHERS',
 'colors': u'Black',
 'dateAdded': 1478857876,
 'dateUpdated': 1478857876,
 'id': u'AVpe_4OLLJeJML430HJo'}